In [2]:
import csv
import numpy as np

In [17]:
privacy_ids = [f"PP_00{i}" for i in range(1, 10)] + [f"PP_0{i}" for i in range(10, 31)]
dignity_ids = [f"D_00{i}" for i in range(1, 10)] + [f"D_0{i}" for i in range(10, 37)]

In [25]:
def gather_scenario_ids(law : str, folder : str = "paperlaws") -> list[str]:
    path = f"data\{folder}_scenarios\moralchoice_{law}_ambiguity.csv"
    results = []
    with open(path, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            results.append(row[0])
    return results

def action_likelihood(action : int, model : str, law : str) -> float:
    # action = 0 for action1, 1 for action2
    if law == "dignity" :
        ids = dignity_ids
    elif law == "privacy" :
        ids = privacy_ids
    else :
        ids = gather_scenario_ids(law)
    likelihoods_r = np.zeros(len(ids))
    likelihoods_no_r = np.zeros(len(ids))
    try :
        path = f"data/responses/new_dp\{law}low\{model}.csv"
    except FileNotFoundError:
        print(f"File not found: {law}/{model}.csv")
        return -1
    for scenario_id in ids:
        actions_r = [0,0]
        actions_no_r = [0,0]
        with open(path, 'r') as file:
            reader = csv.reader(file)
            next(reader)
            for row in reader:
                if row[1] == scenario_id:
                    if row[3] in ['ab_d','compare_d','repeat_d','ab_p','compare_p','repeat_p'] :
                        if 'action1' in row : actions_r[0] += 1
                        elif 'action2' in row : actions_r[1] += 1
                    elif row[3] in ['ab','compare','repeat'] :
                        if 'action1' in row : actions_no_r[0] += 1
                        elif 'action2' in row : actions_no_r[1] += 1
        sum_r = actions_r[0] + actions_r[1]
        sum_no_r = actions_no_r[0] + actions_no_r[1]
        actions_r[0] /= sum_r
        actions_r[1] /= sum_r
        actions_no_r[0] /= sum_no_r
        actions_no_r[1] /= sum_no_r
        likelihoods_r[ids.index(scenario_id)] = actions_r[action]
        likelihoods_no_r[ids.index(scenario_id)] = actions_no_r[action]
    return np.std(likelihoods_r), np.std(likelihoods_no_r)

models = ["google_flan-t5-small", "google_flan-t5-base", "google_flan-t5-large", "bigscience_bloomz-560m", "bigscience_bloomz-1b1", "bigscience_bloomz-1b7", "bigscience_bloomz-3b"]
laws = ["kill", "pain", "disable", "freedom", "pleasure", "deceive", "cheat", "break_promise", "break_law", "duty"]
laws2 = laws + ["privacy","dignity"]

for law in ["privacy", "dignity"]:
    for model in models:
        print(f"Model: {model}, Law: {law}, Action: {0}")
        print(action_likelihood(0, model, law))

Model: google_flan-t5-small, Law: privacy, Action: 0
(0.10453295313490726, 0.10394627151525697)
Model: google_flan-t5-base, Law: privacy, Action: 0
(0.13266249636871114, 0.1200480971612508)
Model: google_flan-t5-large, Law: privacy, Action: 0
(0.047463850886094554, 0.05513200947102995)
Model: bigscience_bloomz-560m, Law: privacy, Action: 0
(0.14782372582591505, 0.1287564527348985)
Model: bigscience_bloomz-1b1, Law: privacy, Action: 0
(0.08213946081165759, 0.11931422720080304)
Model: bigscience_bloomz-1b7, Law: privacy, Action: 0
(0.1284475528980889, 0.10414527892470413)
Model: bigscience_bloomz-3b, Law: privacy, Action: 0
(0.11018293663453348, 0.1084792562331948)
Model: google_flan-t5-small, Law: dignity, Action: 0
(0.0983472233418479, 0.07099145727806946)
Model: google_flan-t5-base, Law: dignity, Action: 0
(0.0755338727238371, 0.07215384427343063)
Model: google_flan-t5-large, Law: dignity, Action: 0
(0.055533999649503854, 0.04968522930241606)
Model: bigscience_bloomz-560m, Law: dignit

In [13]:
for law in ["privacy", "dignity"]:
    for model in models:
        print(f"Model: {model}, Law: {law}, Action: {0}")
        try :
            print(action_likelihood(0, model, law))
        except :
            print("Error")

Model: google_flan-t5-small, Law: privacy, Action: 0
Error
Model: google_flan-t5-base, Law: privacy, Action: 0
Error
Model: google_flan-t5-large, Law: privacy, Action: 0
Error
Model: bigscience_bloomz-560m, Law: privacy, Action: 0
Error
Model: bigscience_bloomz-1b1, Law: privacy, Action: 0
Error
Model: bigscience_bloomz-1b7, Law: privacy, Action: 0
Error
Model: bigscience_bloomz-3b, Law: privacy, Action: 0
Error
Model: google_flan-t5-small, Law: dignity, Action: 0
Error
Model: google_flan-t5-base, Law: dignity, Action: 0
Error
Model: google_flan-t5-large, Law: dignity, Action: 0
Error
Model: bigscience_bloomz-560m, Law: dignity, Action: 0
Error
Model: bigscience_bloomz-1b1, Law: dignity, Action: 0
Error
Model: bigscience_bloomz-1b7, Law: dignity, Action: 0
Error
Model: bigscience_bloomz-3b, Law: dignity, Action: 0
Error


In [ ]:
print(action_likelihood(0, law, model, law))